# Simulating the elastic Marmousi model

_Copyright Lars Gebraad, 2022_ https://github.com/larsgeb/psvWave

In this notebook it is shown how the `psvWave` package is used to simulate the elastic wavefields through a typical model of a geological structure, by using the AGL Elastic Marmousi model. Note that this notebook (and psvWave in general) uses moment tensor sources, instead of volume/force injections.


> Martin, G. S., Wiley, R., and Marfurt, K. J., 2006, Marmousi2: An elastic
upgrade for Marmousi: The Leading Edge, 25, 156–166.
doi:10.1190/1.2172306

In [1]:
import psvWave
import numpy
import matplotlib.pyplot as plt
from scipy import interpolate
import matplotlib
from mpl_toolkits.axes_grid1.axes_divider import make_axes_locatable
import copy
import scipy.signal


font = { "size": 26}
matplotlib.rc("font", **font)


def interpolate_missing_pixels(image, method="nearest", fill_value=0):
    # https://stackoverflow.com/a/68558547/6848887, https://creativecommons.org/licenses/by-sa/4.0/
    
    mask = numpy.ma.masked_invalid(image).mask
    
    h, w = image.shape[:2]
    xx, yy = numpy.meshgrid(numpy.arange(w), numpy.arange(h))
    known_x = xx[~mask]
    known_y = yy[~mask]
    known_v = image[~mask]
    missing_x = xx[mask]
    missing_y = yy[mask]
    interp_values = interpolate.griddata(
        (known_x, known_y),
        known_v,
        (missing_x, missing_y),
        method=method,
        fill_value=fill_value,
    )
    interp_image = image.copy()
    interp_image[missing_y, missing_x] = interp_values
    return interp_image

def divergence(f):
    """
    Computes the divergence of the vector field f, corresponding to dFx/dx + dFy/dy + ...
    :param f: List of ndarrays, where every item of the list is one dimension of the vector field
    :return: Single ndarray of the same shape as each of the items in f, which corresponds to a scalar field
    """
    num_dims = len(f)
    return numpy.ufunc.reduce(numpy.add, [numpy.gradient(f[i], axis=i) for i in range(num_dims)])

Zeros travel time matrix are defined

In [2]:
S1=[10]
S3=[5]
R1=[[50,200,430]]
R3=[[100,200,480]]
t=copy.deepcopy(R1)

First, we load the marmousi model (which was already subsampled x4), and subsample it some further. We don't want to make our simulations too expensive! We will additionally remove the water layer, and use only a cut out.

Note that the original Marmousi model has a grid spacing of 1.25 meters. With the combined x4 x2 subsampling, that leaves a grid spacing of 10 meters.

In [3]:
class medium:
    pass

marmousi = medium()
f=1
nx=450
nz=500
dx=100
dz=100
dt=.01
nt=10000
density=numpy.ones([nx,nz])*1000
s_veloc=numpy.ones([nx,nz])*0
p_veloc=numpy.ones([nx,nz])*1000

marmousi.density = density
marmousi.s_veloc = s_veloc
marmousi.p_veloc = p_veloc

Next, we'll set up the simulations that we want to run. The domain part is mostly defined by the medium. Feel free to tweak the rest of the settings.

In [4]:
settings = psvWave.get_dictionary()

for l in numpy.arange(0,numpy.size(S1),1):
    settings = {
        "domain": {
            "nt": nt,
            "nx_inner": nx,
            "nz_inner": nz,
            "nx_inner_boundary": 0,
            "nz_inner_boundary": 0,
            "dx": dx,  # 5 refers to the original subsampling grid size
            "dz": dz,
            "dt": dt,
        },
        "boundary": {"np_boundary": 50, "np_factor": 0.5e-2},
        "medium": {"scalar_rho": numpy.mean(density), "scalar_vp": numpy.mean(p_veloc), "scalar_vs": numpy.mean(s_veloc)},
        "sources": {
            "peak_frequency": f,
            "n_sources": 1,
            "n_shots": 1,
            "source_timeshift": 0,
            "delay_cycles_per_shot": 0,
            "moment_angles": [90],
            "ix_sources": [S1[l]],
            "iz_sources": [S3[l]],
            "which_source_to_fire_in_which_shot": [[0]],
        },
        "receivers": {
            "nr": numpy.size(R1[l])+1,
            "ix_receivers": [numpy.hstack([R1[l],S1[l]])],
            "iz_receivers": [numpy.hstack([R3[l],S3[l]])],
        },
        "inversion": {"snapshot_interval": 100},
        "basis": {"npx": 1, "npz": 1},
        "output": {"observed_data_folder": ".", "stf_folder": "."},
    }
    
    psvWave.write_dictionary("marmousi.conf",settings)
    solver=psvWave.fdModel("marmousi.conf")
    
    p, s, d = solver.get_parameter_fields()
    
    np = settings["boundary"]["np_boundary"]
    p[:] = numpy.nan
    s[:] = numpy.nan
    d[:] = numpy.nan
    
    p[np:-np, np:-np] = marmousi.p_veloc
    s[np:-np, np:-np] = marmousi.s_veloc
    d[np:-np, np:-np] = marmousi.density

    p = interpolate_missing_pixels(p)
    s = interpolate_missing_pixels(s)
    d = interpolate_missing_pixels(d)
    
    solver.set_parameter_fields(p, s, d)
    
    for i_shot in range(solver.n_shots):
        solver.forward_simulate(i_shot, omp_threads_override=6)
        
    all_snapshots = solver.get_snapshots()
    all_data = solver.get_synthetic_data()
    
    for i in numpy.arange(0,numpy.size(R1[l]),1):
        tt2=numpy.arange(-dt*(nt-2),dt*(nt+1),dt)
        
        tt=all_data[0][0][i][:]
        
        tt3=all_data[0][0][-1][:]
        tt4=scipy.signal.correlate(tt,tt3)
        
        ind=numpy.argmax(tt4)
        t[l][i]=tt2[ind]
    print("\nShot gather",l,"is finished")

Loading configuration file: 'marmousi.conf'.
Parsing passed configuration.


ValueError: Mismatch between nr and receivers.ix_receivers or receivers.iz_receivers

In [ ]:
fig,axes = solver.plot_synthetic_data(exagerration=10)
plt.tight_layout()
fig.set_size_inches(18, 16)

In [ ]:
print(numpy.shape(all_data))

fig, ax = plt.subplots(figsize=(18, 8))
plt.tight_layout()
im0 = plt.imshow(
    all_snapshots[0][0][10][np:-np, np:-np].T, cmap=plt.get_cmap("bone"), extent=[0, nx * dx, 0, nz * dz]
)

In [ ]:
t

In [ ]:
t2=copy.deepcopy(t)
for i in numpy.arange(0,numpy.size(S1),1):
    for j in numpy.arange(0,numpy.size(R1[i]),1):
        t2[i][j]=numpy.sqrt((S1[i]-R1[i][j])**2+(S3[i]-R3[i][j])**2)*dx/numpy.mean(p_veloc)
t2

In [ ]:
e=copy.deepcopy(t)
for i in numpy.arange(0,numpy.size(S1),1):
    e[i]=(numpy.array(t[i])-numpy.array(t2[i])) / numpy.array(t2[i])
e